In [1]:
import cv2 as cv
import numpy as np
from pathlib import Path

In [1]:
def main():
    # --------- USER SETTINGS ----------
    cam_id = 0
    # chessboard INNER corners: (cols, rows)
    pattern_size = (9, 6)   # 9 columns, 6 rows of inner corners
    square_size = 25.0      # millimeters (or any unit you measured)
    desired_size = (1280, 720)
    min_frames = 20
    out_file = Path("c920_calibration.npz")
    # ----------------------------------

    # Termination criteria for cornerSubPix
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 1e-3)

    # Prepare a single "object points" template in 3D (z=0 plane)
    objp = np.zeros((pattern_size[0] * pattern_size[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)
    objp *= square_size

    objpoints = []  # 3D points in real world space
    imgpoints = []  # 2D points in image plane

    cap = cv.VideoCapture(cam_id)
    if not cap.isOpened():
        raise RuntimeError("Cannot open webcam. Try cam_id=1 or check permissions.")

    # Force a fixed resolution (calibrate for the same resolution you’ll use later)
    cap.set(cv.CAP_PROP_FRAME_WIDTH, desired_size[0])
    cap.set(cv.CAP_PROP_FRAME_HEIGHT, desired_size[1])

    print("Instructions:")
    print("- Hold a checkerboard in view. When corners are detected, press SPACE to capture.")
    print("- Press ESC to run calibration (needs at least ~20 good captures).")
    print("- Tip: tilt/rotate the board and move it around the frame for diverse views.\n")

    last_gray_shape = None

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to read frame.")
            break

        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        last_gray_shape = gray.shape[::-1]  # (width, height)

        found, corners = cv.findChessboardCorners(
            gray, pattern_size,
            flags=cv.CALIB_CB_ADAPTIVE_THRESH + cv.CALIB_CB_NORMALIZE_IMAGE)

        vis = frame.copy()
        if found:
            corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            cv.drawChessboardCorners(vis, pattern_size, corners2, found)
            cv.putText(vis, "Corners found. Press SPACE to capture.", (20, 40),
                       cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        else:
            cv.putText(vis, "Show checkerboard...", (20, 40),
                       cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        cv.putText(vis, f"Captured: {len(imgpoints)} (need ~{min_frames})",
                    (20, 80), cv.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        cv.imshow("Calibration", vis)
        key = cv.waitKey(1) & 0xFF

        if key == 27:  # ESC -> calibrate
            break
        elif key == 32 and found:  # SPACE -> store this view
            objpoints.append(objp.copy())
            imgpoints.append(corners2.copy())
            print(f"Captured frame #{len(imgpoints)}")

    cap.release()
    cv.destroyAllWindows()

    if len(imgpoints) < min_frames:
        print(f"Not enough captures: {len(imgpoints)}. Re-run and capture more.")
        return

    image_size = last_gray_shape
    print("\nCalibrating...")

    # Calibrate camera
    ret, K, dist, rvecs, tvecs = cv.calibrateCamera(
        objpoints, imgpoints, image_size, None, None)

    # Compute mean reprojection error (sanity check)
    total_error = 0.0
    total_points = 0
    for i in range(len(objpoints)):
        projected, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], K, dist)
        error = cv.norm(imgpoints[i], projected, cv.NORM_L2)
        n = len(projected)
        total_error += error * error
        total_points += n

    rmse = np.sqrt(total_error / total_points)

    print("\n=== Results ===")
    print("RMS from calibrateCamera:", ret)
    print("Reprojection RMSE (pixels):", rmse)
    print("\nK (camera matrix):\n", K)
    print("\ndist (k1,k2,p1,p2,k3,...):\n", dist)

    np.savez(out_file, K=K, dist=dist, image_size=image_size, rmse=rmse)
    print(f"\nSaved to: {out_file.resolve()}")

In [5]:
main()

Instructions:
- Hold a checkerboard in view. When corners are detected, press SPACE to capture.
- Press ESC to run calibration (needs at least ~20 good captures).
- Tip: tilt/rotate the board and move it around the frame for diverse views.



NameError: name 'cv2' is not defined